# Pair Programming ETL Transformación I

In [2]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [3]:
pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [4]:
df = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col = 0)
df.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

## 1.  Cargaremos los dos ficheros de datos

In [5]:
df2 = pd.read_csv("files/1.paises_meteo_info.csv", index_col = 0)
df2.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [6]:
df2.shape

(64, 17)

## 2. Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
USA
Australia
New Zealand
South Africa
Papua New Guinea

In [7]:
df["country"].unique()

array(['usa', 'australia', 'mexico', 'brazil', 'england', 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'Unknown', 'cuba', 'mauritius',
       'new zealand', 'spain', 'samoa', 'solomon islands', 'japan',
       'egypt', 'st helena, british overseas territory', 'comoros',
       'reunion', 'french polynesia', 'united kingdom',
       'united arab emirates', 'philippines', 'indonesia', 'china',
       'columbia', 'cape verde', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'nigeria', 'tonga', 'scotland',
       'canada', 'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya',
       'russia', 'turks & caicos', 'united 

In [8]:
df_ataques = df[(df["country"]== 'usa') | 
            (df["country"]== "australia") | 
            (df["country"]== "new zealand") |
            (df["country"]== "south africa") |
            (df["country"]== "papua new guinea")
             ]

In [9]:
df_ataques.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [10]:
df_ataques["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [11]:
df_ataques.shape

(4335, 21)

## 3. Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [12]:
df2.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

## 4. Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
4.1 Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [13]:
df2.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [14]:
df2.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea
2,2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,2,11,1011,none,1,0,90,3,Papua New Guinea
3,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,-1,11,1008,none,1,0,85,3,Papua New Guinea
4,4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",26,2,11,1008,none,1,0,85,3,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,180,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 75, 'speed': ...",26,-1,11,1007,rain,3,0,75,3,Papua New Guinea
60,60,183,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 70, 'speed': ...",26,-4,11,1008,rain,3,0,75,3,Papua New Guinea
61,61,186,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 75, 'speed': ...",26,-1,11,1010,rain,3,0,75,4,Papua New Guinea
62,62,189,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 75, 'speed': ...",26,-1,11,1010,rain,3,0,80,3,Papua New Guinea


In [15]:
df2.shape

(64, 17)

In [16]:
df2.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
temp2m                int64
lifted_index          int64
rh2m                  int64
msl_pressure          int64
prec_type            object
prec_amount           int64
snow_depth            int64
wind10m.direction     int64
wind10m.speed         int64
pais                 object
dtype: object

In [17]:
import ast

In [18]:
df2['wind_profile']= df2['wind_profile'].apply(ast.literal_eval)

In [31]:
df2.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,wind_dir_450mb,wind_dir_400mb,wind_dir_350mb,wind_dir_300mb,wind_dir_250mb,wind_dir_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,14,10,7,8,6,-1,-3,-2,-4,-4,-3,-2,-1,-2,-2,-1,100,90,85,80,80,85,75,20,295,300,300,265,140,185,150,220,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,14,14,7,8,5,-1,-3,-3,-4,-4,-2,-2,-1,-3,-1,-1,100,95,85,80,75,80,70,345,280,290,295,275,30,225,165,250,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [20]:
# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna 
# cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. 
# Donde cada columna será key del diccionario o cada elemento de la lista. 

y = df2['wind_profile'].apply(pd.Series)
y.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 100, 'speed': 3}","{'layer': '900mb', 'direction': 90, 'speed': 3}","{'layer': '850mb', 'direction': 85, 'speed': 4}","{'layer': '800mb', 'direction': 80, 'speed': 4}","{'layer': '750mb', 'direction': 80, 'speed': 4}","{'layer': '700mb', 'direction': 85, 'speed': 3}","{'layer': '650mb', 'direction': 75, 'speed': 3}","{'layer': '600mb', 'direction': 20, 'speed': 2}","{'layer': '550mb', 'direction': 295, 'speed': 3}","{'layer': '500mb', 'direction': 300, 'speed': 4}","{'layer': '450mb', 'direction': 300, 'speed': 3}","{'layer': '400mb', 'direction': 265, 'speed': 3}","{'layer': '350mb', 'direction': 140, 'speed': 2}","{'layer': '300mb', 'direction': 185, 'speed': 2}","{'layer': '250mb', 'direction': 150, 'speed': 5}","{'layer': '200mb', 'direction': 220, 'speed': 3}"
1,"{'layer': '950mb', 'direction': 100, 'speed': 3}","{'layer': '900mb', 'direction': 95, 'speed': 4}","{'layer': '850mb', 'direction': 85, 'speed': 4}","{'layer': '800mb', 'direction': 80, 'speed': 4}","{'layer': '750mb', 'direction': 75, 'speed': 4}","{'layer': '700mb', 'direction': 80, 'speed': 3}","{'layer': '650mb', 'direction': 70, 'speed': 3}","{'layer': '600mb', 'direction': 345, 'speed': 2}","{'layer': '550mb', 'direction': 280, 'speed': 3}","{'layer': '500mb', 'direction': 290, 'speed': 4}","{'layer': '450mb', 'direction': 295, 'speed': 3}","{'layer': '400mb', 'direction': 275, 'speed': 3}","{'layer': '350mb', 'direction': 30, 'speed': 2}","{'layer': '300mb', 'direction': 225, 'speed': 3}","{'layer': '250mb', 'direction': 165, 'speed': 4}","{'layer': '200mb', 'direction': 250, 'speed': 2}"
2,"{'layer': '950mb', 'direction': 90, 'speed': 3}","{'layer': '900mb', 'direction': 90, 'speed': 4}","{'layer': '850mb', 'direction': 85, 'speed': 4}","{'layer': '800mb', 'direction': 80, 'speed': 4}","{'layer': '750mb', 'direction': 75, 'speed': 3}","{'layer': '700mb', 'direction': 75, 'speed': 3}","{'layer': '650mb', 'direction': 55, 'speed': 3}","{'layer': '600mb', 'direction': 340, 'speed': 2}","{'layer': '550mb', 'direction': 285, 'speed': 3}","{'layer': '500mb', 'direction': 290, 'speed': 3}","{'layer': '450mb', 'direction': 290, 'speed': 3}","{'layer': '400mb', 'direction': 285, 'speed': 3}","{'layer': '350mb', 'direction': 330, 'speed': 3}","{'layer': '300mb', 'direction': 210, 'speed': 2}","{'layer': '250mb', 'direction': 155, 'speed': 3}","{'layer': '200mb', 'direction': 295, 'speed': 3}"
3,"{'layer': '950mb', 'direction': 90, 'speed': 3}","{'layer': '900mb', 'direction': 85, 'speed': 3}","{'layer': '850mb', 'direction': 85, 'speed': 4}","{'layer': '800mb', 'direction': 70, 'speed': 4}","{'layer': '750mb', 'direction': 65, 'speed': 3}","{'layer': '700mb', 'direction': 65, 'speed': 3}","{'layer': '650mb', 'direction': 30, 'speed': 2}","{'layer': '600mb', 'direction': 320, 'speed': 3}","{'layer': '550mb', 'direction': 280, 'speed': 3}","{'layer': '500mb', 'direction': 275, 'speed': 3}","{'layer': '450mb', 'direction': 280, 'speed': 3}","{'layer': '400mb', 'direction': 270, 'speed': 3}","{'layer': '350mb', 'direction': 315, 'speed': 3}","{'layer': '300mb', 'direction': 195, 'speed': 2}","{'layer': '250mb', 'direction': 150, 'speed': 3}","{'layer': '200mb', 'direction': 315, 'speed': 5}"
4,"{'layer': '950mb', 'direction': 80, 'speed': 3}","{'layer': '900mb', 'direction': 85, 'speed': 4}","{'layer': '850mb', 'direction': 80, 'speed': 4}","{'layer': '800mb', 'direction': 70, 'speed': 4}","{'layer': '750mb', 'direction': 65, 'speed': 3}","{'layer': '700mb', 'direction': 70, 'speed': 3}","{'layer': '650mb', 'direction': 30, 'speed': 2}","{'layer': '600mb', 'direction': 300, 'speed': 3}","{'layer': '550mb', 'direction': 275, 'speed': 3}","{'layer': '500mb', 'direction': 270, 'speed': 3}","{'layer': '450mb', 'direction': 275, 'speed': 4}","{'layer': '400mb', 'direction': 280, 'speed': 5}","{'layer': '350mb', 'direction': 290, 'speed': 3}","{'layer': '300mb', 'direction': 180, 'speed': 2}","{'layer

In [21]:
df2['rh_profile']= df2['rh_profile'].apply(ast.literal_eval)

In [22]:
x = df2["rh_profile"].apply(pd.Series)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 10}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 6}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -1}"
1,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 14}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 5}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -3}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -1}","{'layer': '200mb', 'rh': -1}"
2,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -3}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -1}","{'layer': '200mb', 'rh': -2}"
3,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': -3}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -3}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -2}"
4,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 8}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': -3}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -3}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -3}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': -1}","{'layer': '200mb', 'rh': -2}"


Columna wind, se saca los valores para velocidad y dirección

In [23]:
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(y.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_dir_" + str(y[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(y[i].apply(pd.Series)["direction"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df2.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

In [24]:
df2.head(3)

,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,wind_dir_450mb,wind_dir_400mb,wind_dir_350mb,wind_dir_300mb,wind_dir_250mb,wind_dir_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,100,90,85,80,80,85,75,20,295,300,300,265,140,185,150,220,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,100,95,85,80,75,80,70,345,280,290,295,275,30,225,165,250,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea
2,90,90,85,80,75,75,55,340,285,290,290,285,330,210,155,295,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,2,11,1011,none,1,0,90,3,Papua New Guinea


Columna rh_profile

In [25]:
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)['layer'][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df2.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información.

In [26]:
df2.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,wind_dir_450mb,wind_dir_400mb,wind_dir_350mb,wind_dir_300mb,wind_dir_250mb,wind_dir_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,14,10,7,8,6,-1,-3,-2,-4,-4,-3,-2,-1,-2,-2,-1,100,90,85,80,80,85,75,20,295,300,300,265,140,185,150,220,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,14,14,7,8,5,-1,-3,-3,-4,-4,-2,-2,-1,-3,-1,-1,100,95,85,80,75,80,70,345,280,290,295,275,30,225,165,250,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


Group by

In [27]:
lista_columnas= df2.columns.tolist()
lista_columnas

['rh_950mb',
 'rh_900mb',
 'rh_850mb',
 'rh_800mb',
 'rh_750mb',
 'rh_700mb',
 'rh_650mb',
 'rh_600mb',
 'rh_550mb',
 'rh_500mb',
 'rh_450mb',
 'rh_400mb',
 'rh_350mb',
 'rh_300mb',
 'rh_250mb',
 'rh_200mb',
 'wind_dir_950mb',
 'wind_dir_900mb',
 'wind_dir_850mb',
 'wind_dir_800mb',
 'wind_dir_750mb',
 'wind_dir_700mb',
 'wind_dir_650mb',
 'wind_dir_600mb',
 'wind_dir_550mb',
 'wind_dir_500mb',
 'wind_dir_450mb',
 'wind_dir_400mb',
 'wind_dir_350mb',
 'wind_dir_300mb',
 'wind_dir_250mb',
 'wind_dir_200mb',
 'timepoint',
 'cloudcover',
 'highcloud',
 'midcloud',
 'lowcloud',
 'rh_profile',
 'wind_profile',
 'temp2m',
 'lifted_index',
 'rh2m',
 'msl_pressure',
 'prec_type',
 'prec_amount',
 'snow_depth',
 'wind10m.direction',
 'wind10m.speed',
 'pais']

In [28]:
df_clima= df2.groupby("pais")[lista_columnas].mean().reset_index()

/tmp/ipykernel_492/2351585889.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima= df2.groupby("pais")[lista_columnas].mean().reset_index()


In [29]:
df_clima.head()

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,wind_dir_450mb,wind_dir_400mb,wind_dir_350mb,wind_dir_300mb,wind_dir_250mb,wind_dir_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,Papua New Guinea,13.4375,12.0625,9.375,5.5625,2.125,-0.265625,0.0,0.40625,0.078125,0.53125,2.59375,3.640625,5.078125,4.96875,4.96875,3.015625,81.015625,81.484375,82.734375,82.96875,80.0,70.390625,196.484375,249.375,255.234375,257.109375,255.625,249.921875,245.3125,231.953125,206.953125,220.703125,97.5,4.203125,-9999.0,-9999.0,-9999.0,25.796875,-0.71875,11.21875,1009.953125,2.171875,0.0,82.34375,3.328125


## 5. Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [30]:
df_clima.to_csv("files/2.paises_limpio.csv")